In [2]:
%%time
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logging.getLogger().setLevel(logging.INFO)

from gensim import corpora, models
from collections import defaultdict
import gensim
import Pyro4 


class prob:
        slices = 0
        def __init__(self):
                self.twenty = []
                prob.slices += 1
        def __del__(self):
                prob.slices -= 1
class test:
        def __init__(self):
             self.box = []
def filtered(*args):
        #removed word list    
        read_words = open('stopwords.txt', 'r')
        r_words = read_words.readline()
        #remove common words and tokenize
        stoplist = set(r_words.split())

        texts = [[word for word in document.lower().split() if word not in stoplist]
                  for document in args]
        read_words.close()
        return texts
class iterator1:
    n1 = 0 
    def __init__(self):
        iterator1.n1+=1
    def delete(self):
        iterator1.n1 = 0 
class iterator2:
    n = 0 
    def __init__(self):
        iterator2.n+=1
    def delete(self):
        iterator2.n = 0 
#Add "NULL" to end of each topic to make the comparison string the same length and the unique topic list              
def filler(lists):
    for x in range(25-len(lists)):
            lists.append("NULL\n")
#Outputs 20% side of the abstracts
def outputfile(y): 
        num1 = iterator1()
        number1 = iterator1.n1
        file=open("cu_test_doc"+str(number1)+".txt","w")    
        for line in y:
            file.write(line)
        file.close()
        if number1 == 5:
            num1.delete()
#Create a file for each Directorate and Division 
def outputfile2(y): 
        num = iterator2()
        number = iterator2.n
        file=open("cu_test_dir_div"+str(number)+".txt","w")    
        for line in y:
            file.write(line)
        file.close()
        if number == 5:
            num.delete()                
        

#Directorates and Divisions files
file = open('cu_doc_div.txt', 'r')
dd = file.readlines() 

#Download Abstracts
file = open('cu_tigers.txt', 'r')
dictionary = file.readlines()

FSIZE = len(dictionary)

#split abstract lists length by 5 
t_split = FSIZE / 5
#variable to account for remainder if abstract list length is not evenly divisible by 5 
test_s = FSIZE % 5 

#round value to account for decimals 
r = round(t_split)
r1 = r*2
r2 = (r*2)+r
r3 = (r*4)
r4 = FSIZE - r3

#Objects that hold the abstract array slices 
L20_1 = prob()
L20_2 = prob()
L20_3 = prob()
L20_4 = prob()
L20_5 = prob()

#Objects that hold the Directorate Division 20% groups 
L20_10 = prob()
L20_20 = prob()
L20_30 = prob()
L20_40 = prob()
L20_50 = prob()

#Holders for each group 
t80 = test()
t20 = test()

#20% abstract group storage (first 4)
l1 = L20_1.twenty = dictionary[:r] 
l2 = L20_2.twenty = dictionary[r:r1]  
l3 = L20_3.twenty = dictionary[r1:r2]  
l4 = L20_4.twenty = dictionary[r2:r3]

#make files for 20% abstract groups 
outputfile(l1)
outputfile(l2)
outputfile(l3)
outputfile(l4)

#20% Directorate Division group storage
d1 = L20_10.twenty = dd[:r] 
d2 = L20_20.twenty = dd[r:r1] 
d3 = L20_30.twenty = dd[r1:r2] 
d4 = L20_40.twenty = dd[r2:r3] 

#make files for 20% Directorate Division groups 
outputfile2(d1)
outputfile2(d2)
outputfile2(d3)
outputfile2(d4)
        
# check for last 20% group to account for uneven 5 way split 
if test_s == 0:
    l5 = L20_5.twenty = dictionary[r3:FSIZE]
    d5 = L20_50.twenty = dd[r3:FSIZE] 
    outputfile(l5)
    outputfile2(d5)
else: 
    l5 = L20_5.twenty = dictionary[r3:r4]
    d5 = L20_50.twenty = dd[r3:r4] 
    outputfile(l5)
    outputfile2(d5)
             

#********************Model Training*************************************

t_file = open('topic_count.txt', 'r')
t_count = t_file.read()
TOPIC_COUNT = int(t_count)


#Comparison 1 arrangement, output test group into file, create: dictionary, corpus, lda model
t8a = t80.box = l2 + l3 + l4 + l5 

#create dictionary and filter abstract 
dictionary1 = corpora.Dictionary(filtered(*t8a))
dictionary1.save('/tmp/cu_dict1.dict')

#create corpus
corpus1 = [dictionary1.doc2bow(text) for text in filtered(*t8a)]#filter stop words and covert from document to bag of words
corpora.MmCorpus.serialize('/tmp/cu_corpus1.mm', corpus1)

#create model more passes have better results to an extent, but are more time consuming 
ldamodel1 = gensim.models.ldamodel.LdaModel(corpus1, num_topics=TOPIC_COUNT, id2word = dictionary1, passes=40,distributed=True)

ldamodel1.save('/tmp/CuModel_1.lda')

#Alert when finished running and tell run time 
print("--- %s min ---" % ((time.time() - start_time)/60))

print("done") 


INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(4963 unique tokens: ['vibrant', 'accurately?', 'validates', '8-year', 'supplements']...) from 60 documents (total 12232 corpus positions)
INFO:gensim.utils:saving Dictionary object under /tmp/cu_dict1.dict, separately None
INFO:gensim.corpora.mmcorpus:storing corpus in Matrix Market format to /tmp/cu_corpus1.mm
INFO:gensim.matutils:saving sparse matrix to /tmp/cu_corpus1.mm
INFO:gensim.matutils:PROGRESS: saving document #0
INFO:gensim.matutils:saved 60x4963 matrix, density=3.122% (9297/297780)
INFO:gensim.corpora.indexedcorpus:saving MmCorpus index to /tmp/cu_corpus1.mm.index
INFO:gensim.models.ldamodel:using symmetric alpha at 0.04
INFO:gensim.models.ldamodel:using symmetric eta at 0.04
ERROR:gensim.models.ldamodel:failed to initialize distributed LDA ('gensim.lda_dispatcher')


RuntimeError: failed to initialize distributed LDA ('gensim.lda_dispatcher')

In [ ]:

#Comparison 2 arrangement, output test group into file, create: dictionary, corpus, lda model 
t8b = t80.box = l1 + l3 + l4 + l5 
t2b = t20.box = l2
#create test file
outputfile(l2)
#create dictionary and filter abstract 
dictionary2 = corpora.Dictionary(filtered(*t8b))
dictionary2.save('/tmp/cu_dict2.dict')
#create corpus
corpus2 = [dictionary2.doc2bow(text) for text in filtered(*t8b)]#filter stop words and covert from document to bag of words
corpora.MmCorpus.serialize('/tmp/cu_corpus2.mm', corpus2)
#create model 
ldamodel2 = gensim.models.ldamodel.LdaModel(corpus2, num_topics=25, id2word = dictionary2, passes=5)
ldamodel2.save('/tmp/CuModel_2.lda')


#Comparison 3 arrangement, output test group into file, create: dictionary, corpus, lda model 
t8c = t80.box = l1 + l2 + l4 + l5  
t2c = t20.box = l3
#create test file
outputfile(l3)
#create dictionary and filter abstract 
dictionary3 = corpora.Dictionary(filtered(*t8c))
dictionary3.save('/tmp/cu_dict3.dict')
#create corpus
corpus3 = [dictionary3.doc2bow(text) for text in filtered(*t8c)]#filter stop words and covert from document to bag of words
corpora.MmCorpus.serialize('/tmp/cu_corpus3.mm', corpus3)
#create model 
ldamodel3 = gensim.models.ldamodel.LdaModel(corpus3, num_topics=25, id2word = dictionary3, passes=5)
ldamodel3.save('/tmp/CuModel_3.lda')


#Comparison 4 arrangement, output test group into file, create: dictionary, corpus, lda model
t8d = t80.box = l1 + l2 + l3 + l5 
t2d = t20.box = l4
#create test file
outputfile(l4)
#create dictionary and filter abstract 
dictionary4 = corpora.Dictionary(filtered(*t8d))
dictionary4.save('/tmp/cu_dict4.dict')
#create corpus
corpus4 = [dictionary4.doc2bow(text) for text in filtered(*t8d)]#filter stop words and covert from document to bag of words
corpora.MmCorpus.serialize('/tmp/cu_corpus4.mm', corpus4)
#create model 
ldamodel4 = gensim.models.ldamodel.LdaModel(corpus4, num_topics=25, id2word = dictionary4, passes=5)
ldamodel4.save('/tmp/CuModel_4.lda')

#Comparison 5 arrangement, output test group into file, create: dictionary, corpus, lda model
t8e = t80.box = l1 + l2 + l3 + l4 
t2e = t20.box = l5 
#create test file
outputfile(l5)
#create dictionary and filter abstract 
dictionary5 = corpora.Dictionary(filtered(*t8e))
dictionary5.save('/tmp/cu_dict5.dict')
#create corpus
corpus5 = [dictionary5.doc2bow(text) for text in filtered(*t8e)]#filter stop words and covert from document to bag of words
corpora.MmCorpus.serialize('/tmp/cu_corpus5.mm', corpus5)
#create model 
ldamodel5 = gensim.models.ldamodel.LdaModel(corpus5, num_topics=25, id2word = dictionary5, passes=5)
ldamodel5.save('/tmp/CuModel_5.lda')
